In [ ]:
#Mandatory imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [ ]:
train = pd.read_json("/content/drive/MyDrive/KUBIG/데이터분석 스터디/train.json/data/processed/train.json")
target_train=train['is_iceberg']
test = pd.read_json("/content/drive/MyDrive/KUBIG/데이터분석 스터디/test.json/data/processed/test.json")

In [ ]:
len(train['band_1'][0])

5625

In [ ]:
!pip install keras-layer-normalization

  Preparing metadata (setup.py) ... done
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4654 sha256=ed45ffa953337d2381a7a9f02d48f6fc79fc106898be9ac8c0d794d19df48b3d
  Stored in directory: /root/.cache/pip/wheels/ed/3a/4b/21db23c0cc56c4b219616e181f258eb7c57d36cc5d056fae9a
Successfully built keras-layer-normalization


In [ ]:
train.isna().sum()

id            0
band_1        0
band_2        0
inc_angle     0
is_iceberg    0
dtype: int64

In [ ]:
train.head()

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [ ]:
#target 설정 : is_iceberg
target_train=train['is_iceberg']

In [ ]:
#inc_angle에 NA 확인
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs -> NaN으로 만들어버림.

In [ ]:
#NA를 채우기 pad방식은 바로 이전 데이터를 그대로 복사붙여넣기한다는 뜻.
train['inc_angle']=train['inc_angle'].fillna(method='pad')

In [ ]:
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [ ]:
#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
#왜 reshape를 75,75로 해야 하는거야?? 원래 픽셀이미지가 75*75여서
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2

In [ ]:
#?왜 여기서 band_3을 평균값을 만들었을까? 그냥 하나의 feature를 더 만들어준 것
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)

In [ ]:
X_train.shape

(1604, 75, 75, 3)

In [ ]:
X_train

array([[[[-27.87836 , -27.154118, -27.51624 ],
         [-27.15416 , -29.537888, -28.346024],
         [-28.668615, -31.0306  , -29.849607],
         ...,
         [-25.573483, -29.095308, -27.334396],
         [-26.488674, -29.540878, -28.014776],
         [-30.507013, -27.511766, -29.00939 ]],

        [[-28.66853 , -31.030518, -29.849524],
         [-27.8784  , -33.529335, -30.703869],
         [-27.508776, -31.591175, -29.549976],
         ...,
         [-26.488632, -34.288055, -30.388344],
         [-28.671562, -28.267496, -28.469528],
         [-31.594166, -26.488716, -29.041441]],

        [[-28.66853 , -33.52929 , -31.09891 ],
         [-27.15416 , -37.05116 , -32.10266 ],
         [-25.865042, -31.591175, -28.728107],
         ...,
         [-27.157106, -31.594084, -29.375595],
         [-27.881393, -32.19339 , -30.037392],
         [-32.837124, -29.54092 , -31.189022]],

        ...,

        [[-29.092905, -27.87895 , -28.485928],
         [-28.669163, -27.878992, -28.274078]

In [ ]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1) #배열(array)이나 텐서(tensor)의 마지막 차원(dimension)

In [ ]:
#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from keras.layers import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [ ]:
#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator #데이터증강 시 사용하는 것
batch_size=64 #한 번 iteration에 사용할 샘플의 수를 정의
#즉, 모델을 훈련할 때 한 번에 몇 개의 이미지를 네트워크에 입력할 것인지를 결정
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0, #이미지의 색상 채널 값을 랜덤하게 변경
                         zoom_range = 0.2,
                         rotation_range = 10)

In [ ]:
# generator를 X1,X2에 , X1과 y에 적용하는 것
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
  #gen 자체가 데이터 증강 함수인 거고 gen.flow는 증강한 이미지를 배치로 다시 재정렬하는 느낌인거 아니가?
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55) #.flow는 이미지 데이터를 배치로 모델에 공급
    #fit이랑 비슷한 느낌인
    #근데 왜 (X1,y) (X1,X2) 이렇게 묶은 거임?
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            yield [X1i[0], X2i[1]], X1i[1]

In [ ]:
# Finally create generator #
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]

#EarlyStopping
#정의: EarlyStopping은 모델의 학습을 조기에 종료시키는 콜백
#이 콜백은 특정 메트릭(예: 검증 데이터에 대한 손실 값)이 더 이상 개선되지 않을 때 학습을 멈추게 함
#Patience: patience 매개변수는 지정된 메트릭의 개선이 멈춘 후 몇 에포크까지 학습을 계속 진행할 것인지를 결정
#예를 들어, patience=10은 메트릭이 마지막으로 개선된 이후 10개의 에포크 동안 추가 개선이 없을 경우 학습을 중단하겠다는 것을 의미
#목적: EarlyStopping의 주요 목적은 과적합을 방지하고, 학습 시간을 단축시키기 위함
#모델이 검증 데이터에 대해 더 이상 개선되지 않는다면, 과적합의 위험이 있으므로 학습을 조기에 종료하는 것이 좋음

#정의: ModelCheckpoint 콜백은 학습 중인 모델의 가중치를 저장.
#모델이 더 좋은 성능을 보일 때마다 중간 저장을 할 수 있으며, 학습이 끝난 후에도 최적의 모델을 사용할 수 있음
#save_best_only: save_best_only=True 매개변수는 메트릭이 개선될 때만 모델을 저장하겠다는 것을 의미
# 이 옵션을 사용하면 디스크 공간을 절약하고, 항상 최고의 모델만을 보존 가능

In [ ]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle") #angle의 input은 단일차원 숫자이므로, shape=[1]
    angle_layer = Dense(1, )(input_2) # 위의 angle값을 기반으로 한 Fully Connected Layer -> Dense(1,)
    #(1,)이 들어간 이유는 출력차원이 1이고 angle 입력을 기반으로 해서 1차원을 내보낸다는
    #Dense 레이어의 출력 차원
    #Dense 자체가 Fully connected layer와 같은 말
    base_model = VGG16(weights='imagenet', include_top=False,
                 input_shape=X_train.shape[1:], classes=1)

    x = base_model.get_layer('block5_pool').output
    #shape[1:] -> X_train의 첫 번째 차원(일반적으로 배치 크기를 의미)을 제외한 나머지 차원의 형태를 나타냄
    #예를 들어, X_train이 (배치 크기, 높이, 너비, 채널) 형태의 이미지 데이터라면,
    # X_train.shape[1:]는 (높이, 너비, 채널)을 의미
    #(1604, 75, 75, 3) <<
    #classes=1은 분류할 클래스의 수를 의미,
    #include_top=False -> 마지막 FC레이어를 포함X

    #block5_pool은 VGG16 모델 내의 특정 레이어의 이름으로,
    #이 레이어는 모델의 다섯 번째 블록에서 마지막 풀링 레이어를 의미
    #.output은 해당 레이어의 출력을 나타내며, 이를 통해 block5_pool 레이어의 출력을 다음 모델 구성 단계의 입력으로 사용


#추상화 수준: block5_pool은 VGG16의 마지막 pooling 레이어로, 매우 추상화된 표현을 학습함.

    x = GlobalMaxPooling2D()(x)
    #block5_pool에서도 충분히 특성 추출이 되었지만 maxpooling을 해줘서
    #차원 축소: GlobalMaxPooling2D는 각 특성 맵(feature map)에서 최대값만을 추출하여, 그 결과로 하나의 숫자(최대값)를 반환
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)

    #이 레이어를 통과한 후의 데이터 형태는 (batch_size, height, width, channels)의 4차원 텐서
    #여기서 height와 width는 원본 이미지의 크기가 여러 컨볼루션과 풀링 레이어를 통과하면서 축소된 크기
    #그리고 channels는 특성 맵의 개수로, block5_pool에서는 512개
    #예를 들어, 입력 이미지가 224x224 크기를 가지고 block5_pool을 통과한 후의 텐서 크기는 (batch_size, 7, 7, 512)가 됨
    #여기서 7x7은 이미지의 각 차원이 축소된 크기이며, 512는 특성 맵의 개수

    #GlobalMaxPooling2D는 (batch_size, channels)의 2차원 텐서로 변환
    #이 과정에서 height와 width 차원이 제거되고, 각 특성 맵에서 하나의 최대값만이 남게 됨
    #-> 결국 (batch_size, 512) 형태를 가지게됨
    #이는 각 이미지(배치 내의 각 아이템)에 대해 512개의 최대값을 가지는 벡터가 되며,
    #각 최대값은 해당 특성 맵에서 가장 두드러진 특성의 강도를 나타냄냄

    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)

    predictions = Dense(1, activation='sigmoid')(merge_one)

    model = Model(inputs=[base_model.input, input_2], outputs=predictions) #base_model.input
    sgd = SGD(learning_rate=1e-3, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


In [ ]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx] #shape랑은 관계없이 몇번째 이미지이
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx] #hold: 검증 test
        Y_holdout= target_train[test_idx] #hold: 검증 test

        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx] #hold: 검증 test

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        #%s_aug_model_weights.hdf5 포맷에서 %s는 각 교차 검증 폴드의 인덱스(j)로 대체됨
        #j가 0인 경우, 파일 경로는 0_aug_model_weights.hdf5가 됨
        #=첫 번째 교차 검증 폴드의 모델 가중치를 저장하는 파일명
        #.hdf5 확장자는 Hierarchical Data Format version 5의 약자로, 다양한 유형의 대규모 데이터셋을 저장하기 위한 파일 포맷-> Keras에서는 모델의 구조와 가중치를 이 포맷으로 저장하고 불러오는 것을 지원
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24, #fit_generator 메서드를 사용하여 gen_flow 제너레이터로부터 데이터를 받아 모델을 학습
                #여기서 steps_per_epoch=24는 각 에폭에서 24개의 배치를 사용하여 모델을 학습시킴
                #배치 크기가 64이므로, 한 에폭에서 처리되는 데이터의 총량은 24 * 64
                epochs=100,
                shuffle=True,
                verbose=1, #verbose=1을 설정하면, 모델이 각 에포크를 거칠 때마다 훈련 손실(loss), 검증 손실(validation loss), 훈련 정확도(training accuracy),
                #검증 정확도(validation accuracy)와 같은 주요 메트릭들에 대한 정보를 진행 바와 함께 출력
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0]) #score의 0번째 index가 loss
        print('Train accuracy:', score[1]) #score의 1번째 index가 accuracy
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold]) #X_holdout: X_train[test_indx]
        #galaxyModel.predict([X_holdout, X_angle_hold])은 검증 세트(X_holdout과 X_angle_hold)에 대한 모델의 예측을 수행
        #여기서 X_holdout은 이미지 데이터, X_angle_hold는 해당 이미지들의 추가 정보(예: 각도)를 나타냄
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])
        #pred_valid.reshape(pred_valid.shape[0])은 예측 결과를 1차원 배열로 변환
        #이는 각 예측의 차원을 단순화하여, 각 데이터 포인트에 대한 예측값만을 배열로 갖기 위함
        #첫 번째 차원은 데이터 포인트의 수(또는 배치 크기)를 나타내고,
        #두 번째 차원은 출력 클래스의 확률이나 회귀 결과
        #예측 결과가 [배치 크기, 출력 크기] 형태인 경우, .shape[0]은 배치 내의 샘플 수(데이터 포인트의 수)
        #따라서, 이 값을 사용하여 예측 결과를 1차원 배열로 재구성하거나, 특정 작업을 수행할 때 기준으로 삼을 수 있음
        #즉, pred_log라는 값을 새로 만들 때 shape를 valid를 배치크기라는 1차원 shape로 다시 reshape를 하고 싶은 것


        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])
        #이 예측 결과를 1차원 배열로 변환하고, y_test_pred_log 배열에 누적
        #이 배열은 교차 검증의 모든 폴드에서 테스트 데이터에 대한 예측 결과를 누적하기 위해 사용
        #모든 폴드에 대한 예측을 누적함으로써, 전체 데이터셋에 대한 종합적인 성능 평가를 수행할 수 있음

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K ##누적값이니까 K값으로 나눠줘야 함?
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log


    #여기서
    #Validation score, train score, test score 이 세가지를 모두 측정한 것임!그치만 test_pred_log만 return함

    #train score가 훈련 데이터에 너무 잘 맞으면(훈련 정확도가 매우 높으면), 새로운 데이터에 대한 성능(일반화 성능)이 오히려 떨어질 수 있음
    #훈련 점수와 검증 점수의 차이를 비교함으로써 과적합을 감지할 수 있음

In [ ]:
X_train.shape

(1604, 75, 75, 3)

In [ ]:
X_angle.shape

(1604,)

In [ ]:
target_train.shape

(1604,)

In [ ]:
preds=myAngleCV(X_train, X_angle, X_test)



===================FOLD= 0
58889256/58889256 [==============================] - 0s 0us/step


<ipython-input-22-797144f62d20>:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  galaxyModel.fit_generator(


Epoch 1/100
24/24 [==============================] - ETA: 0s - loss: 0.9848 - accuracy: 0.4759

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - 25s 544ms/step - loss: 0.9848 - accuracy: 0.4759 - val_loss: 0.6447 - val_accuracy: 0.6561
Epoch 2/100
24/24 [==============================] - 6s 249ms/step - loss: 0.6165 - accuracy: 0.6506 - val_loss: 0.8375 - val_accuracy: 0.5869
Epoch 3/100
24/24 [==============================] - 6s 243ms/step - loss: 0.5094 - accuracy: 0.7383 - val_loss: 0.3456 - val_accuracy: 0.8411
Epoch 4/100
24/24 [==============================] - 7s 285ms/step - loss: 0.4034 - accuracy: 0.8154 - val_loss: 0.3237 - val_accuracy: 0.8336
Epoch 5/100
24/24 [==============================] - 5s 230ms/step - loss: 0.3124 - accuracy: 0.8578 - val_loss: 0.2476 - val_accuracy: 0.8991
Epoch 6/100
24/24 [==============================] - 5s 220ms/step - loss: 0.2940 - accuracy: 0.8695 - val_loss: 0.2548 - val_accuracy: 0.8916
Epoch 7/100
24/24 [==============================] - 6s 256ms/step - loss: 0.2749 - accuracy: 0.8807 - val_loss: 0.2517 - val_accuracy: 0.9047
Epoch 8/10

...

24/24 [==============================] - 9s 248ms/step - loss: 0.6906 - accuracy: 0.5913 - val_loss: 0.4458 - val_accuracy: 0.7738
Epoch 2/100
24/24 [==============================] - 7s 297ms/step - loss: 0.4478 - accuracy: 0.7792 - val_loss: 0.3415 - val_accuracy: 0.8187
Epoch 3/100
24/24 [==============================] - 5s 218ms/step - loss: 0.3656 - accuracy: 0.8204 - val_loss: 0.3467 - val_accuracy: 0.8224
Epoch 4/100
24/24 [==============================] - 7s 306ms/step - loss: 0.3269 - accuracy: 0.8477 - val_loss: 0.2589 - val_accuracy: 0.8991
Epoch 5/100
24/24 [==============================] - 6s 275ms/step - loss: 0.2978 - accuracy: 0.8611 - val_loss: 0.2531 - val_accuracy: 0.8953
Epoch 6/100
24/24 [==============================] - 6s 269ms/step - loss: 0.2752 - accuracy: 0.8721 - val_loss: 0.2839 - val_accuracy: 0.8486
Epoch 7/100
24/24 [==============================] - 6s 251ms/step - loss: 0.2691 - accuracy: 0.8820 - val_loss: 0.2224 - val_accuracy: 0.9065
Epoch 8/100

<ipython-input-22-797144f62d20>:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  galaxyModel.fit_generator(


Epoch 1/100
24/24 [==============================] - ETA: 0s - loss: 0.5919 - accuracy: 0.6621

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - 14s 508ms/step - loss: 0.5919 - accuracy: 0.6621 - val_loss: 0.3917 - val_accuracy: 0.8015
Epoch 2/100
24/24 [==============================] - 6s 238ms/step - loss: 0.3969 - accuracy: 0.8090 - val_loss: 0.3430 - val_accuracy: 0.8427
Epoch 3/100
24/24 [==============================] - 6s 259ms/step - loss: 0.3022 - accuracy: 0.8633 - val_loss: 0.2858 - val_accuracy: 0.8727
Epoch 4/100
24/24 [==============================] - 6s 241ms/step - loss: 0.2732 - accuracy: 0.8794 - val_loss: 0.2755 - val_accuracy: 0.8727
Epoch 5/100
24/24 [==============================] - 6s 248ms/step - loss: 0.2443 - accuracy: 0.9040 - val_loss: 0.2778 - val_accuracy: 0.8858
Epoch 6/100
24/24 [==============================] - 5s 219ms/step - loss: 0.2310 - accuracy: 0.8986 - val_loss: 0.2843 - val_accuracy: 0.8820
Epoch 7/100
24/24 [==============================] - 7s 295ms/step - loss: 0.2292 - accuracy: 0.9018 - val_loss: 0.2617 - val_accuracy: 0.9026
Epoch 8/10

In [ ]:
!jupyter nbconvert --to pdf --output "/content/drive/MyDrive/KUBIG/데이터분석 스터디/지윤_iceberg.pdf" "/content/drive/MyDrive/KUBIG/데이터분석 스터디/iceberg".ipynb

[NbConvertApp] Converting notebook /content/drive/MyDrive/KUBIG/데이터분석 스터디/iceberg.ipynb to pdf
[NbConvertApp] Writing 87810 bytes to notebook.tex
[NbConvertApp] Building PDF
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 423, in start
    self.convert_notebooks()
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 597, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 560, in convert_single_notebook
    output, resources = self

In [ ]:
%cd /content/drive/MyDrive/KUBIG/데이터분석 스터디
!jupyter nbconvert --to html --output "/content/drive/MyDrive/KUBIG/데이터분석 스터디/지윤_iceberg.html" "/content/drive/MyDrive/KUBIG/데이터분석 스터디/iceberg.ipynb" #sejda에서 html to pdf 변환

/content/drive/MyDrive/KUBIG/데이터분석 스터디
[NbConvertApp] Converting notebook /content/drive/MyDrive/KUBIG/데이터분석 스터디/iceberg.ipynb to html
[NbConvertApp] Writing 678293 bytes to /content/drive/MyDrive/KUBIG/데이터분석 스터디/지윤_iceberg.html


In [ ]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('sub.csv', index=False)